In [299]:
!pip install tweepy
!pip install textblob
!python -m textblob.download_corpora
!pip install nltk
!pip install numpy

[nltk_data] Downloading package brown to
[nltk_data]     /Users/srinivassuri/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/srinivassuri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/srinivassuri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/srinivassuri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/srinivassuri/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/srinivassuri/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [300]:
# -*- coding: utf-8 -*-
import re
import tweepy
import unicodedata
import json
import nltk
from nltk.stem.porter import *
from nltk.tokenize import TweetTokenizer
import numpy as np

In [301]:
#Class Definiton for the keywords that will be used to fetch the tweets
class keywords:
    def __init__(self):
        self.keywords = ['AlaskaAir','Allegiant','AmericanAir','Delta','FlyFrontier','HawaiianAir']#'United Airlines','Virgin America']
        
    def getKeyWords(self):
        return self.keywords

In [302]:
#Class Definition to pull the twitter tweets
class pullData:      
    def __init__(self,key,secret,maxTweets,tweetsPerQry,keywords):
        self.key = key   
        self.secret = secret
        self.maxTweets = maxTweets
        self.tweetsPerQry = tweetsPerQry          
        self.keywords = keywords
        self.api = ''
        self.auth = ''
        
    def printParams(self):
        print('Parameters set to...')
        print('key...',self.key)
        print('secret...',self.secret)
        print('maxTweets...',self.maxTweets)
        print('tweetsPerQry...',self.tweetsPerQry)
        print('keywords...',self.keywords)
        
    def connect(self):
        self.auth = tweepy.AppAuthHandler(self.key,self.secret )  
        self.api = tweepy.API(self.auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
        return True
        
        if (not self.api):
            print ("Can't Authenticate")
            return False
        
    def downloadData(self):
        for word in self.keywords:
            print( "Downloading Tweets for the keyword: ", word )
            fName = 'tweets_' + word + '.txt'
            sinceId = None
            max_id = -1
            tweetCount = 0
            tweet_dict = []
    
            print("Downloading max {0} tweets".format(maxTweets))

            while tweetCount < self.maxTweets:
                try:
                    if (max_id <= 0):
                        if (not sinceId):   
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry)
                        else:
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry,since_id=sinceId)
                    else:
                        if (not sinceId):
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry,
                                    max_id=str(max_id - 1))
                        else:
                            new_tweets = self.api.search(q=word, count=self.tweetsPerQry,
                                    max_id=str(max_id - 1),since_id=sinceId)
                    if not new_tweets:
                        print("No more tweets found")
                        break            
                        
                    for tweet in new_tweets:
                        tweet_dict.append(tweet._json)
 
                    tweetCount += len(new_tweets)
                    print("Downloaded {0} tweets".format(tweetCount))
                    max_id = new_tweets[-1].id
                
                except tweepy.TweepError as e:
                    # Just exit if any error
                    print("some error : " + str(e))
                    break
            
            with open(fName, 'w') as f:   
                json.dump(tweet_dict, f, ensure_ascii=False)
    

In [ ]:
import sys

consumer_key = 'bppIV0LkQUIARDug4b8Sij8pm'
consumer_secret = '5ofpZtxL1CsDmfbF93Qh0EcPOHUVP3ZSF6CGhbkk2ki2fFWNFL'
maxTweets = 30000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits

keys = keywords()
words = keys.getKeyWords()  # this is what we're searching for

pulldata = pullData(consumer_key,consumer_secret,maxTweets,tweetsPerQry,words)

if ( pulldata.connect() == False ):
    print ( "Connecting to the twitter API Failed")
    sys.exit(1)
pulldata.printParams()
    
pulldata.downloadData()



Parameters set to...
key... bppIV0LkQUIARDug4b8Sij8pm
secret... 5ofpZtxL1CsDmfbF93Qh0EcPOHUVP3ZSF6CGhbkk2ki2fFWNFL
maxTweets... 30000
tweetsPerQry... 100
keywords... ['AlaskaAir', 'Allegiant', 'AmericanAir', 'Delta', 'FlyFrontier', 'HawaiianAir']
Downloaded 100 tweets


In [ ]:
#Class definition for JsonParser

class JsonParser:
    def loadData(self,fname):
        with open(fname) as json_data:
            d = json.load(json_data)
        return d
        

In [ ]:
#Class definition to Clean the tweets and get the sentiment
class TweetCleaner: 
    def __init__(self,stopwords_fname):
        self.stopwords_fname = stopwords_fname
        self.emoji_pattern = re.compile(
            u"(\ud83d[\ude00-\ude4f])|"  # emoticons
            u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
            u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
            u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
            u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
            u"+", flags=re.UNICODE)
        
        #Populate the Stop Words
        self.stopwords = set()
        self.populateStopWords(self.stopwords_fname)
        
        #Initialize nltk classes
        self.stemmer = PorterStemmer()
        self.tokenizer = TweetTokenizer()
    
    #---------Create the Stop words Set---------#
    def populateStopWords(self,fname):
        stop_file = open(fname)
        
        for line in stop_file:
            self.stopwords.add(line.strip())
        
    #---------Clean the tweets---------#
    def cleanTweet(self,tweet):
        tweet = tweet.lower()
        
        #1. remove emojis
        tweet = self.emoji_pattern.sub(r'', tweet)
        tweet_list = self.tokenizer.tokenize(tweet)
        
        #2. remove the stop words
        words_filtered = []
        for word in tweet_list:
            if (word not in self.stopwords ):
                words_filtered.append( word )
              
        #3. Stem the words
        words_stemmed = [self.stemmer.stem(word) for word in words_filtered]
        
        _tweet = ""
        for word in words_stemmed:
            _tweet += " " + word
        
        #4. return the fitered tweet
        return _tweet
             
    #--------- negative: 0 , neutral : 1 , positive : 2 ---------#
    def getSentiment(self,tweet):
        return 2 
        
        

In [ ]:
#Cleans all the tweets , builds the tweet set and adds the sentiment to the tweets

class BuildFeatureSet: 
    
    def __init__(self,n,stopwords_fname,max_features_count=10000): # n : ngram for the tweets
        self.ngrams = n

        #Lexicon related variables
        self.lexicon = {}
        self.inverse_lexicon = { }
        self.ngram_count = 0
        self.max_features_count = max_features_count
        self.tweetCleaner = TweetCleaner(stopwords_fname)
        
        #Training Dataset
        self.data = {}

    def addToLexicon(self,words):
        for word in words:
            if ( word not in self.lexicon and ( self.ngram_count <  self.max_features_count ) ):
                self.lexicon[word] = self.ngram_count #Assign a unique number for the word seen
                self.inverse_lexicon[self.ngram_count] = word
                self.ngram_count = self.ngram_count + 1
                #print( 'Lexicon: ',self.ngram_count,word)
    
    def isTweetReply(self,tweet):
        if ( tweet[0].lower() == 'r' and  tweet[1].lower() == 't' ):
            return True
        else:
            return False
        
    #Add the tweet to the lexicon set
    def addTweet(self,tweet):
        #1. get the sentiment for the tweet
        sentiment = self.tweetCleaner.getSentiment(tweet)
        
        #2. Clean the Tweet
        tweet = self.tweetCleaner.cleanTweet(tweet)
        
        #3. get the ngrams for the tweet
        _ngrams = nltk.ngrams(tweet.split(), self.ngrams)
            
        #4. Add the ngrams to the lexicon dictionary
        words = list(_ngrams)
        self.addToLexicon( words )
        
        #5. Add this tweet row to the training set
        self.addToTrainingData(words,sentiment)
    
    #Build the Feature set for all the tweets
    def addToTrainingData(self,ngrams,sentiment):
        row = np.zeros(self.max_features_count + 1 ) # last feature is the label 
        
        for word in ngrams:
            if ( word in self.lexicon ):
                row[ self.lexicon[word] ] = row[ self.lexicon[word] ] + 1 #Increase the count of the word 

        row [ self.max_features_count ] = sentiment
        
        self.data[ len(self.data)  ]  = row 
    
    
    def getFeatures(self):
        features = np.zeros(( len(self.data), self.max_features_count ))
        
        for index in self.data:
            features[index][:] =  self.data[index][:-1]
            
        return features
    
    def getLabels(self):
        labels = np.zeros( len(self.data) )
        
        for index in self.data:
            labels[index] = ( int(self.data[index][self.max_features_count]) )
            
        return labels
    

In [ ]:
#Parse the Tweets
jsonParser = JsonParser()
features = BuildFeatureSet(1,'stopwords.txt') 

files = ['tweets_Allegiant.txt','tweets_AlaskaAir.txt','tweets_Delta.txt','tweets_AmericanAir.txt','tweets_FlyFrontier.txt','tweets_HawaiianAir.txt'] 


for file in files:
    print ('Building Feature set for the file:',file)
    tweets =  jsonParser.loadData(file)

    for tweet in tweets:
        #1. Skip if the tweet the is a reply to an existing tweet
        if ( buildFeatures.isTweetReply(tweet['text']) == False ):
            features.addTweet ( tweet['text']  ) 

print ( features.getFeatures().shape )

print ( features.getLabels().shape )

